In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [ ]:
#change to appropriate home folder
folder = '/content/drive/MyDrive/pop_pharmacogenomics/gpt/'

In [ ]:
train_labels = glob(folder+'train_xml/*')
drug_label_text = []
for label in tqdm(train_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for section in soup.find_all('Section'):
        section_name = section['name']
        drug_label_text.append([drug_name, section_name, section.text])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'section_name', 'section_text'])
drug_label_text.to_csv(folder+'train_drug_label_text.csv', index=False)
drug_label_text.head()

100%|██████████| 101/101 [00:02<00:00, 44.62it/s]


,drug_name,section_name,section_text
0,KYPROLIS,adverse reactions,6 ADVERSE REACTIONS\n\n The following adv...
1,KYPROLIS,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...
2,MULTAQ,adverse reactions,6 ADVERSE REACTIONS\n\n The following saf...
3,MULTAQ,boxed warnings,\n\n BOXED WARNING: WARNING: INCREASED RISK...
4,MULTAQ,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...


In [ ]:
test_labels = glob(folder+'gold_xml/*')
drug_label_text = []
for label in tqdm(test_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for section in soup.find_all('Section'):
        section_name = section['name']
        drug_label_text.append([drug_name, section_name, section.text])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'section_name', 'section_text'])
drug_label_text.to_csv(folder+'test_drug_label_text.csv', index=False)
drug_label_text.head()

100%|██████████| 99/99 [00:31<00:00,  3.19it/s]


,drug_name,section_name,section_text
0,PREPOPIK,adverse reactions,6 ADVERSE REACTIONS\n\n EXCERPT: Most c...
1,PREPOPIK,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...
2,ICLUSIG,adverse reactions,6 ADVERSE REACTIONS\n\n Because clinical ...
3,ICLUSIG,boxed warnings,\n\n BOXED WARNING: WARNING: VASCULAR OCCLU...
4,ICLUSIG,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...


also extract the manually annotated ades.

In [34]:
train_labels = glob(folder+'train_xml/*')
drug_label_text = []
for label in tqdm(train_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for reaction in soup.find_all('Reaction'):
        string = reaction['str']
        for norm in reaction.find_all('Normalization'):
          try:
            meddra_str = norm['meddra_pt']
            meddra_id = norm['meddra_pt_id']
          except:
            meddra_str, meddra_id = None, None
          try:
            meddra_llt = norm['meddra_llt']
            meddra_llt_id = norm['meddra_llt_id']
          except:
            meddra_llt, meddra_llt_id = None, None
          drug_label_text.append([drug_name, string, meddra_str, meddra_id, meddra_llt, meddra_llt_id])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'reaction_string', 'meddra_pt', 'meddra_pt_id', 'meddra_llt', 'meddra_llt_id'])
drug_label_text.to_csv(folder+'train_drug_label_text_manual_ades.csv', index=False)
drug_label_text.head()

100%|██████████| 101/101 [00:01<00:00, 51.43it/s]


,drug_name,reaction_string,meddra_pt,meddra_pt_id,meddra_llt,meddra_llt_id
0,KYPROLIS,cardiac toxicities,Cardiotoxicity,10048610,None,None
1,KYPROLIS,acute renal failure,Acute kidney injury,10069339,Acute renal failure,10001041
2,KYPROLIS,tumor lysis syndrome,Tumour lysis syndrome,10045170,Tumor lysis syndrome,10045152
3,KYPROLIS,pulmonary toxicity,Pulmonary toxicity,10061924,None,None
4,KYPROLIS,pulmonary hypertension,Pulmonary hypertension,10037400,None,None


In [38]:
test_labels = glob(folder+'gold_xml/*')
drug_label_text = []
for label in tqdm(test_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for reaction in soup.find_all('Reaction'):
        string = reaction['str']
        for norm in reaction.find_all('Normalization'):
          try:
            meddra_str = norm['meddra_pt']
            meddra_id = norm['meddra_pt_id']
          except:
            meddra_str, meddra_id = None, None
          try:
            meddra_llt = norm['meddra_llt']
            meddra_llt_id = norm['meddra_llt_id']
          except:
            meddra_llt, meddra_llt_id = None, None
          drug_label_text.append([drug_name, string, meddra_str, meddra_id, meddra_llt, meddra_llt_id])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'reaction_string', 'meddra_pt', 'meddra_pt_id', 'meddra_llt', 'meddra_llt_id'])
drug_label_text.to_csv(folder+'test_drug_label_text_manual_ades.csv', index=False)
drug_label_text.head()

100%|██████████| 99/99 [00:02<00:00, 46.71it/s]


,drug_name,reaction_string,meddra_pt,meddra_pt_id,meddra_llt,meddra_llt_id
0,PREPOPIK,nausea,Nausea,10028813,None,None
1,PREPOPIK,headache,Headache,10019211,None,None
2,PREPOPIK,vomiting,Vomiting,10047700,None,None
3,PREPOPIK,abdominal bloating,Abdominal distension,10000060,Abdominal bloating,10048746
4,PREPOPIK,abdominal distension,Abdominal distension,10000060,None,None
